# Introduction 
The goal of this notebook is to provide a simply and easy to use tool to extract geometry features from 1D sufraces printed by additive manufacturing methods. This notebook accomplishes the following tasks 
- imports STL data of 1D surfaces (3D scan data required to this)
- Identifies the surface of the 1D object and tresholds the surface to extract the printed track 
- Calculates geometry metrics including track width, height, and volume as well as the roughness metrics (Please not that roughness metrics are hightly dependent on the resolution of the 3D scan data)
- Visualizes and outputs 2D contour images of segmented tracks as well as the appropriate geometry infomation. 

Authorship: Ajay Talbot- PhD Student, University of Toronto, Department of Materials Science and Engineering, Laboratory of Extreme Mechanics and Additive Manufacturing (LEMAM)
email: ajay.talbot@mail.utoronto.ca


# Install appropriate packages
Install the appropriate packages using pip to be able to import all the libraries below. Just uncomment the lines below if you need. 
Depending on what you have installed on your computer, you may need to install more

In [ ]:

# pip install numpy-stl
#pip install numpy matplotlib scikit-learn scipy seaborn pandas



Note: you may need to restart the kernel to use updated packages.


# Geometry Extraction 
the following code is used to extract the geometry of all the STL files. just hit run and it will define all the functions needed 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import struct
import os
from scipy.interpolate import griddata
from scipy.signal import find_peaks
from scipy import stats
import seaborn as sns
import pandas as pd

plt.rcParams.update({
    'font.size': 16,
    'font.family': 'Arial'
})

def read_stl(file_path):
    with open(file_path, 'rb') as file:
        header = file.read(80)
        num_triangles = struct.unpack('<I', file.read(4))[0]
        vertices = []
        volume = 0
        for _ in range(num_triangles):
            file.read(12)  # Normal vector
            v1 = struct.unpack('<fff', file.read(12))
            v2 = struct.unpack('<fff', file.read(12))
            v3 = struct.unpack('<fff', file.read(12))
            vertices.extend([v1, v2, v3])
            file.read(2)  # Attribute byte count

            # Calculate the volume of the triangular prism
            a = np.array(v1)
            b = np.array(v2)
            c = np.array(v3)
            volume += np.abs(np.dot(a, np.cross(b, c))) / 6

        return np.array(vertices), volume


def align_track(points):
    mean = np.mean(points, axis=0)
    centered_points = points - mean
    pca = PCA(n_components=3)
    pca.fit(centered_points)
    principal_axes = pca.components_

    # Ensure the determinant is positive for a right-handed coordinate system
    if np.linalg.det(principal_axes) < 0:
        principal_axes[2] = -principal_axes[2]

    # Assuming the third principal axis should be pointing 'up'
    # Check if it's pointing 'down' instead
    if principal_axes[2, 2] < 0:
        # If the Z component of the third principal axis is negative,
        # it means the track is upside down. Flip it to correct the orientation.
        principal_axes[2] = -principal_axes[2]

    aligned_points = centered_points @ principal_axes.T
    return aligned_points

def create_grid_and_find_max_z(points, grid_size):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    
    # Adding small margins to avoid cutting off the edges
    x_margin = (x_max - x_min) * 0.01
    y_margin = (y_max - y_min) * 0.01
    
    x_grid = np.linspace(x_min - x_margin, x_max + x_margin, grid_size)
    y_grid = np.linspace(y_min - y_margin, y_max + y_margin, grid_size)
    X, Y = np.meshgrid(x_grid, y_grid)
    
    Z = griddata(points[:, :2], points[:, 2], (X, Y), method='nearest')
    
    return X, Y, Z

def find_kde_slope_threshold(z_values, percent):
    sns.kdeplot(z_values, fill=True)
    kde = sns.kdeplot(z_values)
    kde_lines = kde.get_lines()[0]
    x, y = kde_lines.get_data()
    
    # Compute the derivative of the KDE
    dydx = np.gradient(y, x)
    
    # Find peaks in the KDE
    peaks, _ = find_peaks(y)
    if len(peaks) == 0:
        raise ValueError("No peaks found in KDE")
    
    # Filter peaks to consider only those with z values less than 0
    negative_peaks = [peak for peak in peaks if x[peak] < 0]
    if len(negative_peaks) == 0:
        raise ValueError("No peaks found with z values less than 0")
    
    # Identify the largest peak among the negative peaks
    largest_peak_index = max(negative_peaks, key=lambda peak: y[peak])
    peak_value = y[largest_peak_index]
    
    # Find the zero-crossing points after the largest peak
    zero_crossings = np.where(np.diff(np.sign(dydx)))[0]
    zero_crossings_after_peak = zero_crossings[zero_crossings > largest_peak_index]
    if len(zero_crossings_after_peak) == 0:
        raise ValueError("No zero-crossing found past the largest peak")
    
    bottom_peak_index = zero_crossings_after_peak[0]
    bottom_peak_value = x[bottom_peak_index]
    
    # Calculate the Z threshold as a specified percentage between the peak and the bottom of the peak
    z_threshold = x[largest_peak_index] + percent / 100.0 * (bottom_peak_value - x[largest_peak_index])
    
    plt.close()
    return z_threshold


def calculate_surface_roughness(Z):
    # Remove NaNs for calculation
    Z = Z[~np.isnan(Z)]
    
    # Arithmetic Mean Deviation (Ra)
    Ra = np.mean(np.abs(Z - np.mean(Z)))

    # Root Mean Square Deviation (Rq)
    Rq = np.sqrt(np.mean(Z**2))

    # Maximum Profile Peak Height (Rp)
    Rp = np.max(Z)

    # Maximum Profile Valley Depth (Rv)
    Rv = np.min(Z)

    # Ten-Point Mean Roughness (Rz)
    peaks = np.sort(Z)[-5:]
    valleys = np.sort(Z)[:5]
    Rz = np.mean(peaks+valleys)

    # Skewness (Rsk)
    Rsk = (np.mean((Z - np.mean(Z))**3)) / (np.std(Z)**3)

    # Kurtosis (Rku)
    Rku = (np.mean((Z - np.mean(Z))**4)) / (np.std(Z)**4)

    return Ra, Rq, Rp, Rv, Rz, Rsk, Rku

def calculate_widths_and_heights(X, Y, Z, sections=100):
    x_min, x_max = np.min(X), np.max(X)
    section_width = (x_max - x_min) / sections

    widths = []
    heights = []

    for i in range(sections):
        x_start = x_min + i * section_width
        x_end = x_start + section_width

        z_section = Z[(X >= x_start) & (X < x_end)]
        y_section = Y[(X >= x_start) & (X < x_end)]

        if np.any(~np.isnan(z_section)):
            y_min_edge = np.min(y_section[~np.isnan(z_section)])
            y_max_edge = np.max(y_section[~np.isnan(z_section)])

            width = y_max_edge - y_min_edge
            widths.append(width)

            height = np.nanmax(z_section)
            heights.append(height)

    mean_height = np.mean(heights)
    z_scores_heights = stats.zscore(heights)
    
    threshold = 1.5  # Define your outlier threshold
    
    outlier_indices_heights = np.where(abs(z_scores_heights) > threshold)
    outlr_idx_h = len(outlier_indices_heights[0])

    # Calculate surface roughness parameters for the entire Z array
    Ra, Rq, Rp, Rv, Rz, Rsk, Rku = calculate_surface_roughness(Z)

    return widths, heights, np.max(widths), np.max(heights), outlr_idx_h, Ra, Rq, Rp, Rv, Rz, Rsk, Rku

def calculate_volume(X, Y, Z, sections=1000):
    x_min, x_max = np.min(X), np.max(X)
    section_width = (x_max - x_min) / sections

    Volume = 0

    for i in range(sections):
        x_start = x_min + i * section_width
        x_end = x_start + section_width

        z_section = Z[(X >= x_start) & (X < x_end)]
        y_section = Y[(X >= x_start) & (X < x_end)]

        if np.any(~np.isnan(z_section)):
            # Find the Y values at the edges of the contour
            y_min_edge = np.min(y_section[~np.isnan(z_section)])
            y_max_edge = np.max(y_section[~np.isnan(z_section)])

            width = y_max_edge - y_min_edge
            height = np.nanmax(z_section)  # Assuming height is the max Z value in the section

            # Calculate the radius as half of the height
            radius = height / 2

            # Calculate the area of the semi-circle and then the volume of the section
            area = 0.5 * np.pi * height*width*0.5
            volume = area*section_width
            Volume += volume

    return Volume


def plot_projections(file_path, grid_size=1000, percent=75):
    points, volume = read_stl(file_path)
    aligned_points = align_track(points)

    # Find the Z threshold from KDE
    z_values = aligned_points[:, 2]
    z_threshold = find_kde_slope_threshold(z_values, percent)


    X, Y, Z = create_grid_and_find_max_z(aligned_points, grid_size)

    # Apply Z threshold
    Z[Z < z_threshold] = np.nan  # Set values below threshold to NaN
    # Add the absolute value of the z_threshold to each of the filtered points
    Z = np.where(~np.isnan(Z), Z + abs(z_threshold), Z)
    # Calculate widths and heights
    widths, heights, max_width, max_height, outlr_idx_h, Ra, Rq, Rp, Rv, Rz, Rsk, Rku = calculate_widths_and_heights(X, Y, Z)  # Get max width and height
    
    mean_width = np.mean(widths)
    std_width = np.std(widths)
    mean_height = np.mean(heights)
    std_height = np.std(heights)
    Volume = calculate_volume(X, Y, Z)  # Calculate volume
    
    
    # print(f"Mean width: {mean_width}, Std width: {std_width}")
    # print(f"Mean height: {mean_height}, Std height: {std_height}")
    # print(f"Max width: {max_width}, Max height: {max_height}")  # Print max width and height
    stl_file_name = os.path.splitext(os.path.basename(file_path))[0]

#### CONTOUR PLOT WITH LINES
    plt.figure(figsize=(10, 10))
    cp = plt.contour(X, Y, Z, cmap='rainbow', levels=10)
    cbar = plt.colorbar(cp)
    cbar.set_label('Track Height (mm)') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'XY Projection of {os.path.basename(file_path)}')
    plt.axis('equal')

    # Existing code for generating contour lines
    min_contour_level = np.nanmin(Z)
    contour_lines = plt.contour(X, Y, Z, levels=[min_contour_level], colors='none')

    # Check if there is at least one collection and one path
    if contour_lines.collections and contour_lines.collections[0].get_paths():
        path = contour_lines.collections[0].get_paths()[0]
        v = path.vertices
        # Continue with the existing logic using `v`
    else:
        # Handle the situation, e.g., by setting v to an empty array or skipping further processing
        v = np.array([])  # Example of setting `v` to an empty array

    # Calculate the bounding box length in the X plane for the thresholded region
    thresholded_points = aligned_points[aligned_points[:, 2] >= z_threshold]
    x_min_thresholded, x_max_thresholded = np.min(thresholded_points[:, 0]), np.max(thresholded_points[:, 0])
    length = x_max_thresholded - x_min_thresholded

    # Add lines and labels for widths, plotting every 5th line
    x_min, x_max = np.min(X), np.max(X)
    total_sections = 30  # Total sections to plot
    section_width = (x_max - x_min) / total_sections
    for i in range(total_sections):
        section_index = i * (len(widths) // total_sections)  # Calculate the index for every 5th section
        width = widths[section_index]
        x_start = x_min + section_index * (x_max - x_min) / len(widths)
        x_end = x_start + section_width

        # Calculate condition
        condition = (X >= x_start) & (X < x_end) & (~np.isnan(Z))
        filtered_Y = Y[condition]

        # Check if the filtered array is empty
        if filtered_Y.size > 0:
            y_min_edge = np.min(filtered_Y)
            y_max_edge = np.max(filtered_Y)
            plt.plot([x_start, x_start], [y_min_edge, y_max_edge], color='black', linestyle='--')
            plt.plot([x_end, x_end], [y_min_edge, y_max_edge], color='black', linestyle='--')
            plt.text((x_start + x_end) / 2, (y_min_edge + y_max_edge) / 2, f'Width: {width:.2f}', rotation=90)

    y_min, y_max = np.min(thresholded_points[:, 1]), np.max(thresholded_points[:, 1])

    # Draw lines representing the bounding box
    plt.plot([x_min_thresholded, x_min_thresholded], [y_min, y_max], 'r--')  # Left line
    plt.plot([x_max_thresholded, x_max_thresholded], [y_min, y_max], 'r--')  # Right line

    # Annotate the length on the plot
    plt.annotate(f'Length: {length:.2f}', 
                xy=((x_min_thresholded + x_max_thresholded) / 2, y_max), 
                textcoords="offset points", 
                xytext=(0,10), 
                ha='center', 
                arrowprops=dict(arrowstyle="->"))

    plt.savefig(f'{stl_file_name}_output_xy_contour.png', dpi=300)
    plt.close()


    # CONTOUR PLOT FILLED
    cp = plt.contourf(X, Y, Z, cmap='rainbow', levels=10)
    cbar = plt.colorbar(cp)
    cbar.set_label('Track Height (mm)') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'XY Projection of {os.path.basename(file_path)}')
    plt.axis('equal')
    
    plt.savefig(f'{stl_file_name}_output_xy_contour_FILLED.png', dpi=300)
    plt.close()


  # XZ Projection
    plt.figure(figsize=(10, 10))
    plt.scatter(aligned_points[:, 0], aligned_points[:, 2] + abs(z_threshold), s=0.1, c='black')  # Add z_threshold to Z-values
    plt.xlabel('X')
    plt.ylabel('Z')
    plt.title(f'XZ Projection of {os.path.basename(file_path)}')
    plt.axis('equal')
    plt.savefig(f'{stl_file_name}_output_xz.png', dpi=300)
    plt.close()
    

    # KDE of Z data
    plt.figure(figsize=(10, 10))
    sns.kdeplot(aligned_points[:, 2], fill=True)
    plt.axvline(x=z_threshold, color='red', linestyle='--')
    plt.xlabel('Z')
    plt.title(f'KDE of Z data for {os.path.basename(file_path)}')
    plt.savefig(f'{stl_file_name}_output_kde.png', dpi=300)
    plt.close()


    #filtered KDE Plot to check for gaussian distribution
    # plt.figure(figsize=(10, 10))
    # # Filter aligned_points for Z values above the threshold
    # filtered_z_values = aligned_points[aligned_points[:, 2] >= z_threshold][:, 2]
    # sns.kdeplot(filtered_z_values, fill=True)
    # plt.axvline(x=z_threshold, color='red', linestyle='--')
    # plt.xlabel('Z')
    # plt.title(f'KDE of Filtered Z data for {os.path.basename(file_path)}')
    # plt.savefig(f'{stl_file_name}_output_filtered_kde.png', dpi=300)
    # plt.close()


    data = {
        'stl_file_name': [os.path.basename(file_path)],
        'mean_width': [mean_width],
        'std_width': [std_width],
        'max_width': [max_width],
        'mean_height': [mean_height],
        'outlr_idx_h': [outlr_idx_h],
        'std_height': [std_height],
        'max_height': [max_height],         
        'z_threshold': [z_threshold],  # Add Z threshold
        'volume': [Volume],  # Add volume
        'length': [length],  # Add length
        'Ra': [Ra],
        'Rq': [Rq],
        'Rp': [Rp],
        'Rv': [Rv],
        'Rz': [Rz],
        'Rsk': [Rsk],
        'Rku': [Rku]
    }
    df = pd.DataFrame(data)

    return df  # Return max width and height # Return max width and height


# MANUAL THRESHOLDING 
IF THERE IS SOME WEIRD DATA THAT COULDNT BE AUTO THRESHOLDED, YOU CAN MANUALLY THRESHOLD IT BY CHANGING THE z_theshold value in the code below.
Run the function below and then manually change the path to the file you want to change the threshold for. Look at the KDE plot and change the threshold somewhere before 0 but after the peak of the first curve in the plot (indicitive of the substrate)

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import struct
import os
from scipy.interpolate import griddata
from scipy.signal import find_peaks
from scipy import stats
import seaborn as sns
import pandas as pd

plt.rcParams.update({
    'font.size': 16,
    'font.family': 'Arial'
})

def read_stl(file_path):
    with open(file_path, 'rb') as file:
        header = file.read(80)
        num_triangles = struct.unpack('<I', file.read(4))[0]
        vertices = []
        volume = 0
        for _ in range(num_triangles):
            file.read(12)  # Normal vector
            v1 = struct.unpack('<fff', file.read(12))
            v2 = struct.unpack('<fff', file.read(12))
            v3 = struct.unpack('<fff', file.read(12))
            vertices.extend([v1, v2, v3])
            file.read(2)  # Attribute byte count

            # Calculate the volume of the triangular prism
            a = np.array(v1)
            b = np.array(v2)
            c = np.array(v3)
            volume += np.abs(np.dot(a, np.cross(b, c))) / 6

        return np.array(vertices), volume


def align_track(points):
    mean = np.mean(points, axis=0)
    centered_points = points - mean
    pca = PCA(n_components=3)
    pca.fit(centered_points)
    principal_axes = pca.components_

    # Ensure the determinant is positive for a right-handed coordinate system
    if np.linalg.det(principal_axes) < 0:
        principal_axes[2] = -principal_axes[2]

    # Assuming the third principal axis should be pointing 'up'
    # Check if it's pointing 'down' instead
    if principal_axes[2, 2] < 0:
        # If the Z component of the third principal axis is negative,
        # it means the track is upside down. Flip it to correct the orientation.
        principal_axes[2] = -principal_axes[2]

    aligned_points = centered_points @ principal_axes.T
    return aligned_points

def create_grid_and_find_max_z(points, grid_size):
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    
    # Adding small margins to avoid cutting off the edges
    x_margin = (x_max - x_min) * 0.01
    y_margin = (y_max - y_min) * 0.01
    
    x_grid = np.linspace(x_min - x_margin, x_max + x_margin, grid_size)
    y_grid = np.linspace(y_min - y_margin, y_max + y_margin, grid_size)
    X, Y = np.meshgrid(x_grid, y_grid)
    
    Z = griddata(points[:, :2], points[:, 2], (X, Y), method='nearest')
    
    return X, Y, Z

def find_kde_slope_threshold(z_values, percent):
    sns.kdeplot(z_values, fill=True)
    kde = sns.kdeplot(z_values)
    kde_lines = kde.get_lines()[0]
    x, y = kde_lines.get_data()
    
    # Compute the derivative of the KDE
    dydx = np.gradient(y, x)
    
    # Find peaks in the KDE
    peaks, _ = find_peaks(y)
    if len(peaks) == 0:
        raise ValueError("No peaks found in KDE")
    
    # Filter peaks to consider only those with z values less than 0
    negative_peaks = [peak for peak in peaks if x[peak] < 0]
    if len(negative_peaks) == 0:
        raise ValueError("No peaks found with z values less than 0")
    
    # Identify the largest peak among the negative peaks
    largest_peak_index = max(negative_peaks, key=lambda peak: y[peak])
    peak_value = y[largest_peak_index]
    
    # Find the zero-crossing points after the largest peak
    zero_crossings = np.where(np.diff(np.sign(dydx)))[0]
    zero_crossings_after_peak = zero_crossings[zero_crossings > largest_peak_index]
    if len(zero_crossings_after_peak) == 0:
        raise ValueError("No zero-crossing found past the largest peak")
    
    bottom_peak_index = zero_crossings_after_peak[0]
    bottom_peak_value = x[bottom_peak_index]
    
    # Calculate the Z threshold as a specified percentage between the peak and the bottom of the peak
    z_threshold = x[largest_peak_index] + percent / 100.0 * (bottom_peak_value - x[largest_peak_index])
    
    plt.close()
    return z_threshold


def calculate_surface_roughness(Z):
    # Remove NaNs for calculation
    Z = Z[~np.isnan(Z)]
    
    # Arithmetic Mean Deviation (Ra)
    Ra = np.mean(np.abs(Z - np.mean(Z)))

    # Root Mean Square Deviation (Rq)
    Rq = np.sqrt(np.mean(Z**2))

    # Maximum Profile Peak Height (Rp)
    Rp = np.max(Z)

    # Maximum Profile Valley Depth (Rv)
    Rv = np.min(Z)

    # Ten-Point Mean Roughness (Rz)
    peaks = np.sort(Z)[-5:]
    valleys = np.sort(Z)[:5]
    Rz = np.mean(peaks+valleys)

    # Skewness (Rsk)
    Rsk = (np.mean((Z - np.mean(Z))**3)) / (np.std(Z)**3)

    # Kurtosis (Rku)
    Rku = (np.mean((Z - np.mean(Z))**4)) / (np.std(Z)**4)

    return Ra, Rq, Rp, Rv, Rz, Rsk, Rku

def calculate_widths_and_heights(X, Y, Z, sections=100):
    x_min, x_max = np.min(X), np.max(X)
    section_width = (x_max - x_min) / sections

    widths = []
    heights = []

    for i in range(sections):
        x_start = x_min + i * section_width
        x_end = x_start + section_width

        z_section = Z[(X >= x_start) & (X < x_end)]
        y_section = Y[(X >= x_start) & (X < x_end)]

        if np.any(~np.isnan(z_section)):
            y_min_edge = np.min(y_section[~np.isnan(z_section)])
            y_max_edge = np.max(y_section[~np.isnan(z_section)])

            width = y_max_edge - y_min_edge
            widths.append(width)

            height = np.nanmax(z_section)
            heights.append(height)

    mean_height = np.mean(heights)
    z_scores_heights = stats.zscore(heights)
    
    threshold = 1.5  # Define your outlier threshold
    
    outlier_indices_heights = np.where(abs(z_scores_heights) > threshold)
    outlr_idx_h = len(outlier_indices_heights[0])

    # Calculate surface roughness parameters for the entire Z array
    Ra, Rq, Rp, Rv, Rz, Rsk, Rku = calculate_surface_roughness(Z)

    return widths, heights, np.max(widths), np.max(heights), outlr_idx_h, Ra, Rq, Rp, Rv, Rz, Rsk, Rku

def calculate_volume(X, Y, Z, sections=1000):
    x_min, x_max = np.min(X), np.max(X)
    section_width = (x_max - x_min) / sections

    Volume = 0

    for i in range(sections):
        x_start = x_min + i * section_width
        x_end = x_start + section_width

        z_section = Z[(X >= x_start) & (X < x_end)]
        y_section = Y[(X >= x_start) & (X < x_end)]

        if np.any(~np.isnan(z_section)):
            # Find the Y values at the edges of the contour
            y_min_edge = np.min(y_section[~np.isnan(z_section)])
            y_max_edge = np.max(y_section[~np.isnan(z_section)])

            width = y_max_edge - y_min_edge
            height = np.nanmax(z_section)  # Assuming height is the max Z value in the section

            # Calculate the radius as half of the height
            radius = height / 2

            # Calculate the area of the semi-circle and then the volume of the section
            area = 0.5 * np.pi * height*width*0.5
            volume = area*section_width
            Volume += volume

    return Volume


def plot_projections(file_path, grid_size=1000, percent=75, z_threshold=0):
    points, volume = read_stl(file_path)
    aligned_points = align_track(points)

    # Find the Z threshold from KDE
    z_values = aligned_points[:, 2]


    X, Y, Z = create_grid_and_find_max_z(aligned_points, grid_size)

    # Apply Z threshold
    Z[Z < z_threshold] = np.nan  # Set values below threshold to NaN
    # Add the absolute value of the z_threshold to each of the filtered points
    Z = np.where(~np.isnan(Z), Z + abs(z_threshold), Z)
    # Calculate widths and heights
    widths, heights, max_width, max_height, outlr_idx_h, Ra, Rq, Rp, Rv, Rz, Rsk, Rku = calculate_widths_and_heights(X, Y, Z)  # Get max width and height
    
    mean_width = np.mean(widths)
    std_width = np.std(widths)
    mean_height = np.mean(heights)
    std_height = np.std(heights)
    Volume = calculate_volume(X, Y, Z)  # Calculate volume
    
    
    # print(f"Mean width: {mean_width}, Std width: {std_width}")
    # print(f"Mean height: {mean_height}, Std height: {std_height}")
    # print(f"Max width: {max_width}, Max height: {max_height}")  # Print max width and height
    stl_file_name = os.path.splitext(os.path.basename(file_path))[0]

#### CONTOUR PLOT WITH LINES
    plt.figure(figsize=(10, 10))
    cp = plt.contour(X, Y, Z, cmap='rainbow', levels=10)
    cbar = plt.colorbar(cp)
    cbar.set_label('Track Height (mm)') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'XY Projection of {os.path.basename(file_path)}')
    plt.axis('equal')

    # Existing code for generating contour lines
    min_contour_level = np.nanmin(Z)
    contour_lines = plt.contour(X, Y, Z, levels=[min_contour_level], colors='none')

    # Check if there is at least one collection and one path
    if contour_lines.collections and contour_lines.collections[0].get_paths():
        path = contour_lines.collections[0].get_paths()[0]
        v = path.vertices
        # Continue with the existing logic using `v`
    else:
        # Handle the situation, e.g., by setting v to an empty array or skipping further processing
        v = np.array([])  # Example of setting `v` to an empty array

    # Calculate the bounding box length in the X plane for the thresholded region
    thresholded_points = aligned_points[aligned_points[:, 2] >= z_threshold]
    x_min_thresholded, x_max_thresholded = np.min(thresholded_points[:, 0]), np.max(thresholded_points[:, 0])
    length = x_max_thresholded - x_min_thresholded

    # Add lines and labels for widths, plotting every 5th line
    x_min, x_max = np.min(X), np.max(X)
    total_sections = 30  # Total sections to plot
    section_width = (x_max - x_min) / total_sections
    for i in range(total_sections):
        section_index = i * (len(widths) // total_sections)  # Calculate the index for every 5th section
        width = widths[section_index]
        x_start = x_min + section_index * (x_max - x_min) / len(widths)
        x_end = x_start + section_width

        # Calculate condition
        condition = (X >= x_start) & (X < x_end) & (~np.isnan(Z))
        filtered_Y = Y[condition]

        # Check if the filtered array is empty
        if filtered_Y.size > 0:
            y_min_edge = np.min(filtered_Y)
            y_max_edge = np.max(filtered_Y)
            plt.plot([x_start, x_start], [y_min_edge, y_max_edge], color='black', linestyle='--')
            plt.plot([x_end, x_end], [y_min_edge, y_max_edge], color='black', linestyle='--')
            plt.text((x_start + x_end) / 2, (y_min_edge + y_max_edge) / 2, f'Width: {width:.2f}', rotation=90)

    y_min, y_max = np.min(thresholded_points[:, 1]), np.max(thresholded_points[:, 1])

    # Draw lines representing the bounding box
    plt.plot([x_min_thresholded, x_min_thresholded], [y_min, y_max], 'r--')  # Left line
    plt.plot([x_max_thresholded, x_max_thresholded], [y_min, y_max], 'r--')  # Right line

    # Annotate the length on the plot
    plt.annotate(f'Length: {length:.2f}', 
                xy=((x_min_thresholded + x_max_thresholded) / 2, y_max), 
                textcoords="offset points", 
                xytext=(0,10), 
                ha='center', 
                arrowprops=dict(arrowstyle="->"))

    plt.savefig(f'{stl_file_name}_output_xy_contour.png', dpi=300)
    plt.close()


    # CONTOUR PLOT FILLED
    cp = plt.contourf(X, Y, Z, cmap='rainbow', levels=10)
    cbar = plt.colorbar(cp)
    cbar.set_label('Track Height (mm)') 
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'XY Projection of {os.path.basename(file_path)}')
    plt.axis('equal')
    
    plt.savefig(f'{stl_file_name}_output_xy_contour_FILLED.png', dpi=300)
    plt.close()


  # XZ Projection
    plt.figure(figsize=(10, 10))
    plt.scatter(aligned_points[:, 0], aligned_points[:, 2] + abs(z_threshold), s=0.1, c='black')  # Add z_threshold to Z-values
    plt.xlabel('X')
    plt.ylabel('Z')
    plt.title(f'XZ Projection of {os.path.basename(file_path)}')
    plt.axis('equal')
    plt.savefig(f'{stl_file_name}_output_xz.png', dpi=300)
    plt.close()
    

    # KDE of Z data
    plt.figure(figsize=(10, 10))
    sns.kdeplot(aligned_points[:, 2], fill=True)
    plt.axvline(x=z_threshold, color='red', linestyle='--')
    plt.xlabel('Z')
    plt.title(f'KDE of Z data for {os.path.basename(file_path)}')
    plt.savefig(f'{stl_file_name}_output_kde.png', dpi=300)
    plt.close()


    #filtered KDE Plot to check for gaussian distribution
    # plt.figure(figsize=(10, 10))
    # # Filter aligned_points for Z values above the threshold
    # filtered_z_values = aligned_points[aligned_points[:, 2] >= z_threshold][:, 2]
    # sns.kdeplot(filtered_z_values, fill=True)
    # plt.axvline(x=z_threshold, color='red', linestyle='--')
    # plt.xlabel('Z')
    # plt.title(f'KDE of Filtered Z data for {os.path.basename(file_path)}')
    # plt.savefig(f'{stl_file_name}_output_filtered_kde.png', dpi=300)
    # plt.close()


    data = {
        'stl_file_name': [os.path.basename(file_path)],
        'mean_width': [mean_width],
        'std_width': [std_width],
        'max_width': [max_width],
        'mean_height': [mean_height],
        'outlr_idx_h': [outlr_idx_h],
        'std_height': [std_height],
        'max_height': [max_height],         
        'z_threshold': [z_threshold],  # Add Z threshold
        'volume': [Volume],  # Add volume
        'length': [length],  # Add length
        'Ra': [Ra],
        'Rq': [Rq],
        'Rp': [Rp],
        'Rv': [Rv],
        'Rz': [Rz],
        'Rsk': [Rsk],
        'Rku': [Rku]
    }
    df = pd.DataFrame(data)

    return df  # Return max width and height # Return max width and height


In [32]:
######## CHANGE THE FILE PATH TO THE STL FILE YOU WANT TO MANUALLY THRESHOLD AND THEN MANUALLY CHANGE THE "z_threshold" VALUE TO THE DESIRED VALUE
file_path = '/Users/ajaytalbot/Library/CloudStorage/OneDrive-UniversityofToronto/STL Files/Single Tracks/TrackINSS5.stl'
plot_projections(file_path, grid_size=1000, percent=60, z_threshold=-0.5)

/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_11589/647104666.py:258: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  if contour_lines.collections and contour_lines.collections[0].get_paths():
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_11589/647104666.py:259: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  path = contour_lines.collections[0].get_paths()[0]


,stl_file_name,mean_width,std_width,max_width,mean_height,outlr_idx_h,std_height,max_height,z_threshold,volume,length,Ra,Rq,Rp,Rv,Rz,Rsk,Rku
0,TrackINSS5.stl,1.768647,0.0,1.768647,0.740944,10,0.14797,0.86909,-0.5,11.849888,11.435175,0.21988,0.495239,0.86909,0.115717,0.984807,0.373294,1.520487


# Loop for Geometry Extraction
Make sure to specify the directort path where the STL files are located. Code will then run and extract everything needed.

In [166]:
import os
import pandas as pd  # Ensure pandas is imported

directory = '/Users/ajaytalbot/Library/CloudStorage/OneDrive-UniversityofToronto/STL Files/Single Tracks'
results = []

for filename in os.listdir(directory):
    if filename.endswith(".stl"):
        try:
            file_path = os.path.join(directory, filename)
            result = plot_projections(file_path, grid_size=1000, percent=60)
            results.append(result)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

    df = pd.concat(results, ignore_index=True)



/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_29983/2458407878.py:259: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  if contour_lines.collections and contour_lines.collections[0].get_paths():
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_29983/2458407878.py:260: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  path = contour_lines.collections[0].get_paths()[0]
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_29983/2458407878.py:259: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  if contour_lines.collections and contour_lines.collections[0].get_paths():
/var/folders/by/cl20h8994bq9jtq1mdk23hmc0000gn/T/ipykernel_29983/2458407878.py:260: MatplotlibDeprecationWarning: The collections attrib

## Dataframe Creation and Export
Run these lines to make sure the data is being displayed properly. 

In [167]:
df_geom=df

In [168]:
df_geom

,stl_file_name,mean_width,std_width,max_width,mean_height,outlr_idx_h,std_height,max_height,z_threshold,volume,length,Ra,Rq,Rp,Rv,Rz,Rsk,Rku
0,Track9.stl,0.858855,0.084374,0.950184,0.202202,6,0.048240,0.274716,-0.052163,18.778051,10.575848,0.058161,0.150720,0.274716,0.000014,0.274730,-0.189105,2.057495
1,TrackSS21.stl,0.884881,0.098286,0.952421,0.446752,7,0.075672,0.496298,-0.097529,29.944098,10.672044,0.095844,0.371376,0.496298,0.000615,0.496914,-1.214499,3.601690
2,TrackSS35.stl,1.074922,0.147051,1.158333,0.484809,7,0.091851,0.551265,-0.164682,160.370421,10.666943,0.105937,0.403602,0.551265,0.000101,0.551365,-1.083703,3.333452
3,Track15.stl,0.937902,0.111427,1.281946,0.277670,10,0.051889,0.374157,-0.069800,16.973001,10.908054,0.072236,0.210692,0.374157,0.000012,0.374169,-0.362616,2.369195
4,Track29.stl,0.901708,0.102640,1.024702,0.229821,7,0.047087,0.314887,-0.063285,61.845191,10.677670,0.062860,0.173320,0.314887,0.000063,0.314950,-0.349188,2.172694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Track24.stl,0.964819,0.106960,1.022998,0.426148,8,0.083003,0.493173,-0.138023,76.891976,10.692512,0.105115,0.345753,0.493173,0.000174,0.493347,-0.860856,2.701697
116,Track18.stl,0.824978,0.134755,1.582406,0.187927,10,0.042778,0.260749,-0.029368,24.209671,10.555745,0.052139,0.141798,0.260749,0.000002,0.260751,-0.134815,2.224697
117,TrackSS38.stl,0.979619,0.118788,1.073118,0.370715,6,0.065438,0.421525,-0.079425,146.107009,10.685210,0.088757,0.295816,0.421525,0.000295,0.421820,-0.895900,2.746738
118,TrackSS10.stl,0.894410,0.087280,0.969864,0.394161,6,0.063934,0.442626,-0.085149,61.726204,10.652258,0.092864,0.316347,0.442626,0.000212,0.442837,-1.009157,2.948823


### WARNING 
Please keep in mind that the code does not run files in order. You will have to order the files in ascending order afterwards.

In [173]:
#export the dataframes to csv files
df_geom.to_csv('df_track_4.csv', index=False)

